# Тестовое задание для стажёра на backend

## Задание:
Определить наиболее пригодный для жилья район в г. Москва на основе открытых данных https://data.mos.ru

### Задача min:
1. Получить данные локально
2. Научиться по адресу определять район в г.Москве
3. Рассчитать метрику качества жизни в некотором районе

Район = Административный округ (АО)

In [1]:
import numpy as np
import pandas as pd

## 1. Получим данные из открытых источников

__Источники:__

1. Общественное питание в г.Москва
    * https://data.mos.ru/opendata/7710881420-obshchestvennoe-pitanie-v-moskve
2. Поликлиники в г. Москва
    * взрослые: https://data.mos.ru/opendata/7707089084-poliklinicheskaya-pomoshch-vzroslym
    * дети: https://data.mos.ru/opendata/7707089084-poliklinicheskaya-pomoshch-detyam/
3. Образовательные учреждения
    * https://data.mos.ru/opendata/7719028495-obrazovatelnye-uchrejdeniya-goroda-moskvy
4. Покрытие городским WIFI
    * https://data.mos.ru/opendata/7710878000-gorodskoy-wi-fi
5. Данные о вызовах пожарно-спасательного гарнизона в г. Москва
    * https://data.mos.ru/opendata/7710474791-dannye-vyzovov-pojarnoy-slujby-po-ao-goroda-moskvy

In [2]:
# Данные о точках общественого питания
data_1_food = pd.read_json("data/data_1_food.json", encoding='windows-1251')

# Данные о поликлиниках для взрослых
data_2_med_adults = pd.read_json("data/data_2_med_adults.json", encoding='windows-1251')

# Данные о детских поликлиниках
data_2_med_childrens = pd.read_json("data/data_2_med_childrens.json", encoding='windows-1251')

# Данные об образовательных учреждениях
data_3_schools = pd.read_json("data/data_3_school.json", encoding='windows-1251')

# Данные о точках бесплатного доступа к сети WiFi
# data_4_WiFi = pd.read_json("data/data_4_wifi.json", encoding='windows-1251')

# Данные о вызовах пожарно-спасательного гарнизона
data_5_fire = pd.read_json("data/data_5_fire.json", encoding='windows-1251')

Посмотрим, как выглядят и что содержат полученные данные.

In [3]:
data_1_food.head(1)

,global_id,Latitude_WGS84,ID,Name,IsNetObject,TypeObject,AdmArea,District,Address,PublicPhone,SeatsCount,SocialPrivileges,Longitude_WGS84,geoData,OperatingCompany
0,637376221,55.879002,151635,СМЕТАНА,нет,кафе,Северо-Восточный административный округ,Ярославский район,"город Москва, улица Егора Абакумова, дом 9",[{'PublicPhone': '(499) 183-14-10'}],48,нет,37.714565,"{'type': 'Point', 'coordinates': [37.714565000...",NaN


In [4]:
data_1_food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16483 entries, 0 to 16482
Data columns (total 15 columns):
global_id           16483 non-null int64
Latitude_WGS84      16483 non-null float64
ID                  16483 non-null int64
Name                16483 non-null object
IsNetObject         16483 non-null object
TypeObject          16483 non-null object
AdmArea             16483 non-null object
District            16483 non-null object
Address             16483 non-null object
PublicPhone         16483 non-null object
SeatsCount          16483 non-null int64
SocialPrivileges    16483 non-null object
Longitude_WGS84     16483 non-null float64
geoData             16483 non-null object
OperatingCompany    3398 non-null object
dtypes: float64(2), int64(3), object(10)
memory usage: 1.9+ MB


In [5]:
data_2_med_adults.head(1)

,global_id,DoctorsSpecialtiesList,AmbulanceStationPhone,OrgInfo,ObjectAddress,FullName,ShortName,DoctorsSpecialtiesListForSelfRecording,Category,AgeRestriction,...,Email,CloseFlag,WorkingHours,DrugStore,AmbulanceStation,ClarificationWorkingHours,PaidServiceInfo,Specialization,BeneficialDrugPrescription,Extrainfo
0,656107910,"[Врач-аллерголог-иммунолог, Врач-гастроэнтерол...",[{'AmbulancePhone': '(495) 841-85-03'}],"[{'OGRN': '1035011454741', 'ChiefPosition': 'и...","[{'PostalCode': '108823', 'AdmArea': 'Новомоск...",Государственное бюджетное учреждение здравоохр...,ГБУЗ Больница «Кузнечики» ДЗМ. Рязановская пол...,"[Врач-акушер-гинеколог, Врач общей практики, В...",Поликлиника взрослая,взрослое,...,[{'Email': 'bk@zdrav.mos.ru'}],действует,"[{'WorkHours': '08:00-20:00', 'DayWeek': 'поне...",да,да,NaN,NaN,NaN,NaN,NaN


In [6]:
data_2_med_adults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 26 columns):
global_id                                 276 non-null int64
DoctorsSpecialtiesList                    276 non-null object
AmbulanceStationPhone                     276 non-null object
OrgInfo                                   276 non-null object
ObjectAddress                             276 non-null object
FullName                                  276 non-null object
ShortName                                 276 non-null object
DoctorsSpecialtiesListForSelfRecording    276 non-null object
Category                                  276 non-null object
AgeRestriction                            134 non-null object
ChiefName                                 276 non-null object
ChiefPosition                             276 non-null object
ChiefGender                               276 non-null object
ChiefPhone                                276 non-null object
PublicPhone                 

In [7]:
data_2_med_childrens.head(1)

,global_id,AmbulanceStation,FullName,ShortName,DoctorsSpecialtiesListForSelfRecording,Category,AgeRestriction,AmbulanceStationPhone,OrgInfo,ObjectAddress,...,Fax,Email,CloseFlag,WorkingHours,ClarificationWorkingHours,DrugStore,BedSpace,PaidServiceInfo,Specialization,Extrainfo
0,656107926,нет,Государственное бюджетное учреждение здравоохр...,ГБУЗ Больница «Кузнечики» ДЗМ.,"[Врач-педиатр, Врач-педиатр участковый, Врач-д...",Поликлиника детская,детское,[{'AmbulancePhone': '(495) 841-85-03'}],"[{'OGRN': '1035011454741', 'ChiefPosition': 'и...","[{'PostalCode': '108824', 'AdmArea': 'Новомоск...",...,[{'Fax': '(495) 212-14-80'}],[{'Email': 'bk@zdrav.mos.ru'}],действует,"[{'WorkHours': '08:00-20:00', 'DayWeek': 'поне...",Воскресенье с 9.00 до 15.00 - обслуживание выз...,да,NaN,NaN,NaN,NaN


In [8]:
data_2_med_childrens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 25 columns):
global_id                                 208 non-null int64
AmbulanceStation                          208 non-null object
FullName                                  208 non-null object
ShortName                                 208 non-null object
DoctorsSpecialtiesListForSelfRecording    208 non-null object
Category                                  208 non-null object
AgeRestriction                            177 non-null object
AmbulanceStationPhone                     208 non-null object
OrgInfo                                   208 non-null object
ObjectAddress                             208 non-null object
ChiefName                                 208 non-null object
ChiefPosition                             208 non-null object
ChiefGender                               208 non-null object
ChiefPhone                                208 non-null object
PublicPhone                 

In [9]:
data_3_schools.head(1)

,global_id,EntranceExaminationsAndUniforms,KPP,LicensingAndAccreditation,OGRN,LegalOrganization,Subordination,ChiefName,LegalAddress,PublicPhone,...,WebSite,EducationPrograms,EducationalServices,ReorganizationStatus,IDEKIS,FullName,ShortName,InstitutionsAddresses,INN,Number
0,658727748,[],773401001,"[{'LicenseSeries': '77Л01', 'LicenseNumber': '...",1037739525273,бюджетное,Городское подчинение ДОНМ,Ледовская Татьяна Леонидовна,"123154, ГОРОД МОСКВА, УЛИЦА МАРШАЛА ТУХАЧЕВСКО...",[{'PublicPhone': '(499) 197-51-23'}],...,dtim.mskobr.ru,Дополнительное образование | дополнительное об...,[],действующее,13072,Государственное бюджетное образовательное учре...,ГБОУ ДО ДТДМ «Хорошево»,"[{'ChiefName': 'Ледовская Татьяна Леонидовна',...",7734077088,NaN


In [10]:
data_3_schools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602 entries, 0 to 601
Data columns (total 21 columns):
global_id                          602 non-null int64
EntranceExaminationsAndUniforms    602 non-null object
KPP                                602 non-null int64
LicensingAndAccreditation          602 non-null object
OGRN                               602 non-null int64
LegalOrganization                  602 non-null object
Subordination                      602 non-null object
ChiefName                          602 non-null object
LegalAddress                       602 non-null object
PublicPhone                        602 non-null object
Email                              602 non-null object
WebSite                            602 non-null object
EducationPrograms                  602 non-null object
EducationalServices                602 non-null object
ReorganizationStatus               602 non-null object
IDEKIS                             602 non-null int64
FullName           

In [11]:
data_5_fire.head()

,global_id,Calls,AdmArea,Year,Month,ID
0,37153160,408,Центральный административный округ,2015,Январь,1
1,37153161,408,Северный административный округ,2015,Январь,2
2,37153162,445,Северо-Восточный административный округ,2015,Январь,3
3,37153163,619,Восточный административный округ,2015,Январь,4
4,37153164,541,Юго-Восточный административный округ,2015,Январь,5


In [12]:
data_5_fire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 6 columns):
global_id    704 non-null int64
Calls        704 non-null int64
AdmArea      704 non-null object
Year         704 non-null int64
Month        704 non-null object
ID           704 non-null int64
dtypes: int64(4), object(2)
memory usage: 33.1+ KB


## 2. Научимся определять район Москвы по адресу

*Переформулирую задачу, как понял её я:*

Необходимо подготовить данные для анализа. Данные обо всех учреждениях собрать по административным округам.

Данные будем собирать в переменную __district_data__. Объединять с новыми данными будем по полю __AdmArea__

### Данные о точках общественого питания

Эти данные имеют столбец __AdmArea__ с указанием административных округов. Сгруппируем по ним все данные и посчитаем количество организаций общественного питания.

In [13]:
district_data = data_1_food.groupby('AdmArea', as_index=False).agg({'Name': 'count'}) \
                .rename(columns={'Name': 'food'})

In [14]:
district_data

,AdmArea,food
0,Восточный административный округ,1314
1,Западный административный округ,1380
2,Зеленоградский административный округ,243
3,Новомосковский административный округ,599
4,Северный административный округ,1486
5,Северо-Восточный административный округ,1565
6,Северо-Западный административный округ,764
7,Троицкий административный округ,162
8,Центральный административный округ,4483
9,Юго-Восточный административный округ,1142


### Данные о поликлиниках

__Поликлиники для взрослого населения__. Информация об административных округов была найдена в столбце __ObjectAddress__ в поле __AdmArea__. Для каждой поликлиники создадим отдельный столбец с указанием АО, в котором поликлиники расположены.

In [15]:
data_2_med_adults['AdmArea'] = np.array([data_2_med_adults.ObjectAddress[i][0]['AdmArea']
                                         for i in range(data_2_med_adults.shape[0])])
tmp = data_2_med_adults.groupby('AdmArea', as_index=False).agg({'FullName': 'count'}) \
        .rename(columns={'FullName': 'med_adults'})

district_data = district_data.join(tmp.set_index('AdmArea'), on='AdmArea')

In [16]:
district_data

,AdmArea,food,med_adults
0,Восточный административный округ,1314,29
1,Западный административный округ,1380,26
2,Зеленоградский административный округ,243,5
3,Новомосковский административный округ,599,18
4,Северный административный округ,1486,24
5,Северо-Восточный административный округ,1565,25
6,Северо-Западный административный округ,764,18
7,Троицкий административный округ,162,14
8,Центральный административный округ,4483,26
9,Юго-Восточный административный округ,1142,29


__Детские поликлиники__. Данные по расположению аналогичны данным по поликлиникам для взрослого населения. Действия будут те же.

In [17]:
data_2_med_childrens['AdmArea'] = np.array([data_2_med_childrens.ObjectAddress[i][0]['AdmArea']
                                         for i in range(data_2_med_childrens.shape[0])])
tmp = data_2_med_childrens.groupby('AdmArea', as_index=False).agg({'FullName': 'count'}) \
        .rename(columns={'FullName': 'med_childrens'})

district_data = district_data.join(tmp.set_index('AdmArea'), on='AdmArea')

In [18]:
district_data

,AdmArea,food,med_adults,med_childrens
0,Восточный административный округ,1314,29,24
1,Западный административный округ,1380,26,20
2,Зеленоградский административный округ,243,5,4
3,Новомосковский административный округ,599,18,15
4,Северный административный округ,1486,24,17
5,Северо-Восточный административный округ,1565,25,20
6,Северо-Западный административный округ,764,18,14
7,Троицкий административный округ,162,14,6
8,Центральный административный округ,4483,26,16
9,Юго-Восточный административный округ,1142,29,26


__Объединим данные__ о поликлиниках в один столбец, поскольку в этом решении нет необходимости использовать данные раздельно.

In [19]:
district_data['med'] = district_data.med_adults + district_data.med_childrens
district_data.drop(['med_adults', 'med_childrens'], axis=1, inplace=True)

In [20]:
district_data

,AdmArea,food,med
0,Восточный административный округ,1314,53
1,Западный административный округ,1380,46
2,Зеленоградский административный округ,243,9
3,Новомосковский административный округ,599,33
4,Северный административный округ,1486,41
5,Северо-Восточный административный округ,1565,45
6,Северо-Западный административный округ,764,32
7,Троицкий административный округ,162,20
8,Центральный административный округ,4483,42
9,Юго-Восточный административный округ,1142,55


### Данные об образовательных учреждениях

Информация об административных округов была найдена в столбце __InstitutionsAddresses__ в поле __AdmArea__. Аналогично данным о поликлиниках создадим отдельный столбец __AdmArea__.

In [21]:
data_3_schools['AdmArea'] = np.array([data_3_schools.InstitutionsAddresses[i][0]['AdmArea']
                                      for i in range(data_3_schools.shape[0])])
tmp = data_3_schools.groupby('AdmArea', as_index=False).agg({'FullName': 'count'}) \
        .rename(columns={'FullName': 'school'})

district_data = district_data.join(tmp.set_index('AdmArea'), on='AdmArea')

In [22]:
district_data

,AdmArea,food,med,school
0,Восточный административный округ,1314,53,63
1,Западный административный округ,1380,46,68
2,Зеленоградский административный округ,243,9,18
3,Новомосковский административный округ,599,33,11
4,Северный административный округ,1486,41,68
5,Северо-Восточный административный округ,1565,45,58
6,Северо-Западный административный округ,764,32,51
7,Троицкий административный округ,162,20,3
8,Центральный административный округ,4483,42,62
9,Юго-Восточный административный округ,1142,55,53


### Данные о вызовах пожарно-спасательного гарнизона

Эти данные содержат информацию не только за 2019 год, поэтому исключим лишние записи.

In [23]:
data_5_fire = data_5_fire[data_5_fire.Year == 2019]

Поскольку здесь информация уже разбита по административным округам, нам остаётся только сгруппировать данные по АО и просуммировать количество вызовов.

In [24]:
data_5_fire = data_5_fire.groupby('AdmArea', as_index=False).agg({'Calls': 'sum'}) \
        .rename(columns={'Calls': 'fire_calls'})

In [25]:
data_5_fire

,AdmArea,fire_calls
0,Восточный административный округ,8821
1,Западный административный округ,7990
2,Зеленоградский административный округ,2244
3,Северный административный округ,6681
4,Северо-Восточный административный округ,7126
5,Северо-Закпадный административный округ,415
6,Северо-Западный административный округ,4438
7,Северо-Заподный административный округ,520
8,Троицкий и Новомосковский административные округа,4649
9,Троицкий и Новомосковский административный округа,762


Заметим, что эти данные имеют ошибки в написании АО. Исправим их.

In [26]:
data_5_fire.iloc[6, 1] = data_5_fire.iloc[5, 1] + data_5_fire.iloc[6, 1] + data_5_fire.iloc[7, 1]
data_5_fire.iloc[5, 1] = 0
data_5_fire.iloc[7, 1] = 0

data_5_fire.iloc[8, 1] = data_5_fire.iloc[8, 1] + data_5_fire.iloc[9, 1]
data_5_fire.iloc[9, 1] = 0

data_5_fire = data_5_fire[data_5_fire.fire_calls > 0].reset_index(drop=True)

In [27]:
data_5_fire

,AdmArea,fire_calls
0,Восточный административный округ,8821
1,Западный административный округ,7990
2,Зеленоградский административный округ,2244
3,Северный административный округ,6681
4,Северо-Восточный административный округ,7126
5,Северо-Западный административный округ,5373
6,Троицкий и Новомосковский административные округа,5411
7,Центральный административный округ,6204
8,Юго-Восточный административный округ,9087
9,Юго-Западный административный округ,6772


Теперь все строки записаны правильно. Но возникла проблема с новообразованными административными округами столицы: в отличие от __district_data__, в __data_5_fire__ Троицкий (ТАО) и Новомосковский (НАО) административные округа записаны в одной строке. Поскольку недостаточно данных о вызовах пожарно-спасательного гарнизона по каждому из этих округов отдельно, объединим ТАО и НАО в __district_data__.

In [28]:
district_data.loc[district_data.shape[0]] = np.array(["Троицкий и Новомосковский административные округа", 
                                                      district_data.iloc[3, 1] + district_data.iloc[7, 1],
                                                      district_data.iloc[3, 2] + district_data.iloc[7, 2],
                                                      district_data.iloc[3, 3] + district_data.iloc[7, 3]
                                                     ])

In [29]:
district_data = district_data.join(data_5_fire.set_index('AdmArea'), on='AdmArea')

In [30]:
# Удалим строки с неизвестными данными, а также изменим тип значений на целочисленный.
district_data = district_data.dropna().astype({'food': 'int',
                                               'med': 'int',
                                               'school': 'int',
                                               'fire_calls': 'int'
                                              }).reset_index(drop=True)

In [31]:
district_data

,AdmArea,food,med,school,fire_calls
0,Восточный административный округ,1314,53,63,8821
1,Западный административный округ,1380,46,68,7990
2,Зеленоградский административный округ,243,9,18,2244
3,Северный административный округ,1486,41,68,6681
4,Северо-Восточный административный округ,1565,45,58,7126
5,Северо-Западный административный округ,764,32,51,5373
6,Центральный административный округ,4483,42,62,6204
7,Юго-Восточный административный округ,1142,55,53,9087
8,Юго-Западный административный округ,1367,55,73,6772
9,Южный административный округ,1978,53,74,9927


### 3. Рассчитаем метрику качества жизни в районах

$$ Качество\ жизни = \frac{n_{поликлиники} + n_{ОУ} + n_{питание}}{относительная\ опасность\ района} $$

$$ относительная\ опасность\ района = \frac{количество\ вызовов\ в\ районе}{общее\ количество\ вызовов\ за\ год} $$

Здесь $n$ - агрегированное по району значение некоторого показателя, ОУ - образовательные учреждения.

Пусть агрегированными значениями показателей будут отношения количества организаций к максимальному количеству организаций по каждому показателю.

In [32]:
district_data.food = district_data.food / np.max(district_data.food)
district_data.med = district_data.med / np.max(district_data.med)
district_data.school = district_data.school / np.max(district_data.school)

Теперь данные в __district_data__ готовы для расчёта качества жизни.

In [33]:
# общее количество вызовов пожарно-спасательного гарнизона за год
sum_fire = np.sum(district_data.fire_calls)

In [34]:
# Качество жизни
district_data['quality'] = (district_data.med + district_data.school + district_data.food) / \
    (district_data.fire_calls / sum_fire)

In [35]:
# Посмотрим, как выглядит рейтинг административных округов с коэффициентами
district_data.sort_values('quality', ascending=False).reset_index(drop=True)

,AdmArea,food,med,school,fire_calls,quality
0,Центральный административный округ,1.000000,0.763636,0.837838,6204,31.715845
1,Юго-Западный административный округ,0.304930,1.000000,0.986486,6772,25.592669
2,Северный административный округ,0.331474,0.745455,0.918919,6681,22.595114
3,Северо-Восточный административный округ,0.349097,0.818182,0.783784,7126,20.708748
4,Северо-Западный административный округ,0.170422,0.581818,0.689189,5373,20.291070
5,Западный административный округ,0.307830,0.836364,0.918919,7990,19.530106
6,Троицкий и Новомосковский административные округа,0.169752,0.963636,0.189189,5411,18.487249
7,Южный административный округ,0.441222,0.963636,1.000000,9927,18.323149
8,Восточный административный округ,0.293107,0.963636,0.851351,8821,18.075941
9,Юго-Восточный административный округ,0.254740,1.000000,0.716216,9087,16.405332


#### Ответ на поставленную задачу:

In [36]:
result = district_data.sort_values('quality', ascending=False).reset_index(drop=True).iloc[0, 0]

print("Наиболее благоприятный административный округ для проживания в городе Москва:", result);

Наиболее благоприятный административный округ для проживания в городе Москва: Центральный административный округ
